# Derotator Axis

This notebook calculates the position of the derotator rotation axis in detector coordinates.

First, take an image of perhaps 15 seconds of a star field. After the first 5 seconds, rotate the derotator with the hand paddle. This will give an image with two images of each star. To rotate the derotator in remote mode, use 6 on the hand controller and the up and down buttons.

Second, measure the coordinates (x0, y0), and (x1, y1) of pairs of images with imexam. Always move from one image to the next in the same sense (e.g., the more counterclockwise image followed by the more clockwise image).

Third, create the array "data" below containing [x0, y0, x1, y1] values.

Finally, run the notebook. This will fit for the center and angle of rotation. Check the RMS and individual deviations. Remove bad data, and iterate as needed. 

The notebook also prints the derotator-centered windows for the TCS config file.

In [6]:
data = [
    # I think this data is from June 2024 with the OGSE
    [ 1400.62, 3518.56,  427.29, 1695.39 ],
    [  784.95, 2999.50,  833.85, 1000.66 ],
    [ 1933.44,  990.66, 3009.39, 1790.85 ],
#    [ 3217.29, 1411.64, 2812.86, 3125.63 ],
    [ 1835.40,  555.63, 3421.12, 1619.99 ],
#    [ 1854.16, 3185.49,  832.60, 2084.25 ],
    [ 2331.88, 2317.47, 1769.46, 2408.31 ],
    [ 2253.92, 1714.96, 2350.31, 2228.64 ],
]

data = [
    # 20250920T043905C1o.fits
    [ 2004.37, 2321.63, 2131.76, 2205.12 ],
    [ 2005.05, 2082.50, 1895.42, 2168.23 ],
    [ 2308.83, 2248.10, 2104.85, 1893.16 ],
    [ 2287.41, 2247.97, 2101.46, 1914.31 ],
    [ 2289.91, 1548.97, 1410.45, 1806.66 ],
    [ 2039.92, 1442.91, 1268.00, 2037.81 ],
    [ 1128.15, 2357.37, 2036.03, 3076.34 ],
    [  972.46, 2224.17, 1881.00, 3210.03 ],
    [ 2921.97, 2210.14, 2159.07, 1281.51 ],
    [ 1796.54, 1042.85,  835.90, 2218.17 ],
]

In [7]:
import math

def rotate(x, y, xc, yc, theta):
    dx = x - xc
    dy = y - yc
    newdx = dx * math.cos(theta) - dy * math.sin(theta)
    newdy = dy * math.cos(theta) + dx * math.sin(theta)
    newx = xc + newdx
    newy = yc + newdy
    return newx, newy

def calcrms(args):
    sumsq = 0
    for x0, y0, x1, y1 in data:
        modelx1, modely1 = rotate(x0, y0, *args)
        dx = x1 - modelx1
        dy = y1 - modely1
        sumsq += dx * dx + dy * dy
    rms = math.sqrt(sumsq / (2 * len(data)))
    return rms

def report(args):
    sumsq = 0
    for x0, y0, x1, y1 in data:
        modelx1, modely1 = rotate(x0, y0, *args)
        dx = x1 - modelx1
        dy = y1 - modely1
        print("%7.2f %7.2f %7.2f %7.2f %7.2f %7.2f %+7.2f %+7.2f" % (x0, y0, x1, y1, modelx1, modely1, dx, dy))
    print("rms = %.2f" % calcrms(args))
    
import scipy.optimize

res = scipy.optimize.minimize(calcrms, [2048, 2048, math.radians(65)])

print(res)
print(res.x)
report(res.x)

xc = res.x[0]
yc = res.x[1]
print("Center of rotation is (%.1f, %.1f)." % (xc, yc))

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.28362049048335064
        x: [ 2.000e+03  2.189e+03 -1.420e+00]
      nit: 18
      jac: [ 6.814e-06  1.144e-06  2.697e-04]
 hess_inv: [[ 1.299e+00  1.834e+00  9.896e-04]
            [ 1.834e+00  3.006e+00  1.604e-03]
            [ 9.896e-04  1.604e-03  1.902e-06]]
     nfev: 312
     njev: 75
[ 2.00016951e+03  2.18917981e+03 -1.42024206e+00]
2004.37 2321.63 2131.76 2205.12 2131.75 2204.89   +0.01   +0.23
2005.05 2082.50 1895.42 2168.23 1895.43 2168.35   -0.01   -0.12
2308.83 2248.10 2104.85 1893.16 2104.72 1892.85   +0.13   +0.31
2287.41 2247.97 2101.46 1914.31 2101.38 1914.01   +0.08   +0.30
2289.91 1548.97 1410.45 1806.66 1410.66 1806.69   -0.21   -0.03
2039.92 1442.91 1268.00 2037.81 1268.30 2037.95   -0.30   -0.14
1128.15 2357.37 2036.03 3076.34 2035.67 3076.56   +0.36   -0.22
 972.46 2224.17 1881.00 3210.03 1880.62 3210.51   +0.38   -0.48
2921.97 2210.14 2159.07 128

## Windows

These are the derotator-centered windows for the TCS config file.

In [ ]:
# These are the sx and sy values for the window with which the rotated images
# were taken. The default 4kx4k window for DDRAGO has sx0 = 2 and sy0 = 8.

sx0 = 2
sy0 = 8

for n in [256, 512, 1024, 2*1024, 3*1024]:
    print("\"derotator-%dx%d\": { \"sx\": %.0f, \"sy\": %.0f, \"nx\": %.0f, \"ny\": %.0f }" % (n, n, sx0 + xc - n / 2, sy0 + yc - n / 2, n, n))

"derotator-256x256": { "sx": 1874, "sy": 2069, "nx": 256, "ny": 256 }
"derotator-512x512": { "sx": 1746, "sy": 1941, "nx": 512, "ny": 512 }
"derotator-1024x1024": { "sx": 1490, "sy": 1685, "nx": 1024, "ny": 1024 }
"derotator-2048x2048": { "sx": 978, "sy": 1173, "nx": 2048, "ny": 2048 }
"derotator-3072x3072": { "sx": 466, "sy": 661, "nx": 3072, "ny": 3072 }
